In [1]:
import numpy as np
import torch
from torch import nn
import copy
import imp
import numpy as np

from sktensor  import dtensor, ktensor, cp_als

import matplotlib.pylab as plt
%matplotlib inline

In [9]:
import sys
sys.path.append('../')

from model_utils import load_model, DATA_ROOT, SAVE_ROOT

MODEL_NAME = 'vgg16_imagenet'
# MODEL_NAME = 'resnet50_imagenet'

model = load_model(MODEL_NAME)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

### BTD layer

In [4]:
import sys
sys.path.append('../')
from tensor_compression.compress import btd2_init_random, btd2_als, btd1_init_random, btd1_als 

class BTDTucker2DecomposedLayer(nn.Module):
    def __init__(self,  layer, layer_name, tucker_ranks = None):
        super(BTDTucker2DecomposedLayer, self).__init__()
        self.tucker_ranks = tucker_ranks
        self.btd_rank = len(self.tucker_ranks[0])
        
        self.layer_name = layer_name
        
        if layer._get_name() != 'Conv2d':
            raise AttributeError('only convolution layer can be decomposed')
        self.cin = layer.in_channels
        self.cout = layer.out_channels

        self.kernel_size = layer.kernel_size
        self.padding = layer.padding
        self.stride = layer.stride
        print('cin : {}, cout : {}, kernel : {}'.format(self.cin, self.cout, self.kernel_size))
        
        
        
        self.weight = layer.weight.data
        self.weight = self.weight.reshape(*self.weight.shape[:2], -1)
        try:
            self.bias = layer.bias.data
        except:
            self.bias = None
        
        print('Create..............')
        self.create_decomposed_layers()
        
        print('Initialize...........')
        self.iniialize_decomposed_layers()
     
        self.weight = None
        self.bias = None
        
    def create_decomposed_layers(self):
        for k in range(self.btd_rank):
            branch = nn.Sequential()
            for j, l in enumerate(self.create_branch(k)):
                branch.add_module('{}'.format(j), l)
            
            self.add_module('{}-{}'.format(self.layer_name, k), branch)
        
        self.add_module('{}-{}'.format(self.layer_name, k+1), nn.ReLU(inplace = True))

            
    def iniialize_decomposed_layers(self):
        for k, (weights, biases) in enumerate(list(zip(*self.get_factors()))):
        
            for j, (w, b)  in enumerate(zip(weights, biases)):
                self.__getattr__('{}-{}'.format(self.layer_name, k)).__getattr__('{}'.format(j)).weight.data = w
               
                if b is not None:
                    self.__getattr__('{}-{}'.format(self.layer_name, k)).__getattr__('{}'.format(j)).bias.data = b
                else:
                    self.__getattr__('{}-{}'.format(self.layer_name, k)).__getattr__('{}'.format(j)).bias = None 

    
#     def get_weights_to_decompose(self):
#         weight = layer.weight.data
#         weight = weight.reshape(*weight.shape[:2], -1)
#         try:
#             bias = layer.bias.data
#         except:
#             bias = None
#         return weight, bias
        
        
    def get_factors(self):
        bias = self.bias
        
        cores, As, Bs = btd2_init_random(self.weight.shape, self.tucker_ranks)
        guess = [cores, As, Bs]
        cores, As, Bs = btd2_als(self.weight, guess, return_fit = False)
        
      
        cores  = [core.numpy() for core in cores]
        As = [A.numpy() for A in As]
        Bs = [B.numpy() for B in Bs]
        
        for k, (core, A, B) in enumerate(zip(cores, As, Bs)):
            print('k = {}, core: {}, A : {}, B : {}'.format(k, core.shape, A.shape, B.shape))
        
        w_cins = []
        w_cores = []
        w_couts = []
        
        for k in range(self.btd_rank):
            w_cin = torch.FloatTensor(np.reshape(Bs[k].T,
                                                 [self.tucker_ranks[k][1], self.cin, 1, 1])).contiguous()
            w_core = torch.FloatTensor(np.reshape(cores[k],
                                                  [self.tucker_ranks[k][0], self.tucker_ranks[k][1], *self.kernel_size])).contiguous()
            w_cout = torch.FloatTensor(np.reshape(As[k], [self.cout, self.tucker_ranks[k][0], 1, 1])).contiguous()
            
            w_cins.append(w_cin)
            w_cores.append(w_core)
            w_couts.append(w_cout)
            
        
        biases  = [[None, None,  None]]*(self.btd_rank-1) + [[None, None,  bias]]
        
        return list(zip(*[w_cins, w_cores, w_couts])),  biases
    
    
    def create_branch(self, k):
        layers = [] 
        layers.append(nn.Conv2d(in_channels=self.cin,
                                    out_channels=self.tucker_ranks[k][1],
                                    kernel_size = (1, 1)))

        layers.append(nn.Conv2d(in_channels = self.tucker_ranks[k][1], 
                                    out_channels=self.tucker_ranks[k][0],
                                    kernel_size = self.kernel_size,
                                    groups = 1, 
                                    padding = self.padding,
                                    stride = self.stride))

        layers.append(nn.Conv2d(in_channels = self.tucker_ranks[k][0],
                                    out_channels = self.cout, 
                                    kernel_size = (1, 1)))
        return layers
    
    
        
    def forward(self, x):
        for i,l in enumerate(list(self.children())[:-1]):
            if i == 0:
                out = l(x)
            else:
                out += l(x)
        
        out = list(self.children())[-1](out)
        
        x = out
        return x

Using numpy backend.
Using pytorch backend.


In [10]:
layer_name = '5'
layer = model.features.__getattr__(layer_name)

R1 = [6, 8]
R2 = [2, 4]
R = list(zip(R1, R2))
print(R)


module_name = 'features'
decomposed_layer = BTDTucker2DecomposedLayer(model.__getattr__(module_name).__getattr__(layer_name),
                                               layer_name, R)

compressed_model = copy.deepcopy(model)
compressed_model.__getattr__(module_name).__setattr__(layer_name, decomposed_layer)



[(6, 2), (8, 4)]
cin : 64, cout : 128, kernel : (3, 3)
Create..............
Initialize...........
k = 0, core: (6, 2, 9), A : (128, 6), B : (64, 2)
k = 1, core: (8, 4, 9), A : (128, 8), B : (64, 4)


In [11]:
compressed_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): BTDTucker2DecomposedLayer(
      (5-0): Sequential(
        (0): Conv2d(64, 2, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): Conv2d(2, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): Conv2d(6, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (5-1): Sequential(
        (0): Conv2d(64, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): Conv2d(8, 128, kernel_size=(1, 1), stride=(1, 1))
      )
      (5-2): ReLU(inplace)
    )
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding

### Create data loaders

In [6]:
# Choose from 'mnist', 'cifar10', 'imagenet'
DATASET = 'imagenet'

##### Load data
import dataloaders

bs = 8

loaders = dataloaders.get_loader(bs, DATASET, DATA_ROOT, num_workers = 0, simple_normalize=True)
test_loader = loaders['val']

Building imagenet data loader with 0 workers


In [12]:
batch = dataloaders.get_batch(test_loader)
batch.shape

torch.Size([8, 3, 224, 224])

In [13]:
compressed_model(batch)

tensor([[ 3.7639,  4.6316,  3.2578,  ..., -2.6185,  1.8284,  2.0557],
        [-0.4243,  0.7162,  3.1696,  ..., -3.1432,  0.3533,  1.0447],
        [ 0.1949,  0.8280, -0.3820,  ..., -2.8338,  1.8218,  4.0224],
        ...,
        [-0.8698,  0.3646,  1.2397,  ..., -2.1741,  2.7798,  1.5710],
        [-1.5034,  0.5579,  1.0676,  ..., -4.8371, -0.3656, -0.6902],
        [ 2.7631,  3.4931,  3.3120,  ..., -3.7741,  0.6654,  0.8105]],
       grad_fn=<AddmmBackward>)